In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm

In [2]:
# 這邊是要把要做的 List 塞進去
# 再來是設定 Output 檔案 
# !! (一定要有反斜線做結尾) !!
# 原本資料       => 日期 牙齒
# 原 Label 資料  => 日期 牙齒_Label
# Trainning資料  => 日期 牙齒_Trainning
DoOrgList =[
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor/",
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor2/",
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor3/",
]

LabelOrgList = [
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor_Label/",
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor2_Label/",
    "D:/Dentist/Data/ScanData/2018.10.18/20181016_Incisor3_Label/",
]

PicMinNumber = 60
PicMaxNumber = 200
RowCutIndex = 50
CutIndex = 500
RadiusBorder = 3

# 人工 Input 的 & 最後 Trainning 要用的
InputPath = "_InputBorder3Class"
TrainningPath = "_TrainningBorder3Class"

# 這邊是 SegNet 相關的設定
TrainTxtLocation = "D:/Dentist/Data/ScanData/2018.10.18/train.txt"

WindowsDataLocation = "D:/Dentist/Data/ScanData/2018.10.18/"
UbuntuDataLocation = "/home/cgal/Desktop/DentistData/2018.11.13_Border_Test_3Class/"

In [3]:
# 把資料夾的名稱 Append 某個 String
def FileNameAppend(FileName, Type):
    return FileName[:-1] + Type + "/"

In [4]:
# 後面會用到的變數
Org_FileNameList = []
Label_FileNameList = []
Input_FileNameList = []
Trainning_FileNameList= []

OrgImageList = []
LabelImageList = []

def CheckIsFileMissing():
    global Org_FileNameList, Label_FileNameList, Input_FileNameList, Trainning_FileNameList
    global OrgImageList, LabelImageList
    
    # Check 有沒有漏轉檔案 & 加進要處理的 List 中
    for i in range(len(DoOrgList)):
        for picIndex in range(PicMinNumber, PicMaxNumber + 1):
            # 路徑
            OrgFileName = DoOrgList[i] + str(picIndex) + ".png"
            LabelFileName = LabelOrgList[i] + str(picIndex) + ".png"
            
            # 0 ~ 250 可以拆成 5 個
            for sliceIndex in range(5):
                InputFileName = FileNameAppend(DoOrgList[i], InputPath) + str(picIndex) + "_" + str(sliceIndex) + ".png"
                TrainningFileName = FileNameAppend(DoOrgList[i], TrainningPath) + str(picIndex) + "_" + str(sliceIndex) + ".png"
                
                Input_FileNameList.append(InputFileName)
                Trainning_FileNameList.append(TrainningFileName)
                
            # 圖片
            imgOrg  = cv2.imread(OrgFileName, cv2.IMREAD_GRAYSCALE)
            imgLabel = cv2.imread(LabelFileName, cv2.IMREAD_GRAYSCALE)

            if(imgOrg is None):
                print(OrgFileName)
                return True
            
            if(imgLabel is None):
                print(LabelFileName)
                return True
            
            # 加到 List 中
            Org_FileNameList.append(OrgFileName)
            Label_FileNameList.append(LabelFileName)
            
            # 圖片 List
            OrgImageList.append(imgOrg)
            LabelImageList.append(imgLabel)
    return False

In [5]:
print("有漏失" if CheckIsFileMissing() else "沒有漏失")

沒有漏失


In [6]:
len(OrgImageList)

423

In [7]:
row, col = OrgImageList[0].shape
print(row)
print(col)
print(OrgImageList[0].dtype)

250
1024
uint8


In [8]:
assert row % RowCutIndex == 0, "RowCutIndex 要整除 Row"
rowOffset = int(row / RowCutIndex)

## 資料轉換
---
1. 要先把資料 Resize
2. 圖片中白的 => 0
3. 圖片中黑的 => 1

In [9]:
# 假設 Trainning 的資料夾沒有建好
# 會重建一個
for i in range(len(DoOrgList)):
    if(not os.path.exists(FileNameAppend(DoOrgList[i], TrainningPath))):
        os.mkdir(FileNameAppend(DoOrgList[i], TrainningPath))
    
    if(not os.path.exists(FileNameAppend(DoOrgList[i], InputPath))):
        os.mkdir(FileNameAppend(DoOrgList[i], InputPath))

In [10]:
# 判斷這個點的半徑內，是不是邊界
def CheckIfIsBorder(Image, rowIndex, colIndex):
    global row, col, RadiusBorder
    minRow = max(0,   rowIndex - RadiusBorder)
    maxRow = min(row, rowIndex + RadiusBorder)
    minCol = max(0,   colIndex - RadiusBorder)
    maxCol = min(col, colIndex + RadiusBorder)
    
    value = Image[rowIndex][colIndex]
    for r in range(minRow, maxRow):
        for c in range(minCol, maxCol):
            if Image[r][c] != value:
                return True
    return False

In [11]:
# 這邊是只輸出一張 BGR 的圖片
def BGRWithBorder():
    global Org_FileNameList, Label_FileNameList, Input_FileNameList, Trainning_FileNameList
    global OrgImageList, LabelImageList
    global CutIndex, RowCutIndex, rowOffset
    
    # 這邊是要刪除的 List，以防寫出檔案的時候有錯誤
    deleteIndex = []
    
    for i in tqdm(range(len(Org_FileNameList))):
        # 每個 200 張 要跳過不做
        if (Org_FileNameList[i][-7:] == "200.png" or Org_FileNameList[i][-7:] == "199.png"):
            for j in range(rowOffset):
                deleteIndex.append(i * rowOffset + j)
            continue
        
        # 改圖片
        for sliceIndex in range(5):
            InputImageBGR = np.zeros((RowCutIndex, min(col, CutIndex), 3), dtype=np.uint8)       # Input 
            ResultImageBGR = np.zeros((RowCutIndex, min(col, CutIndex)), dtype=np.uint8)         # Result
            
            for r in range(RowCutIndex):
                for c in range(min(col, CutIndex)):
                    for colorIndex in range(3):
                        # 原圖直接丟回去
                        # B G R
                        InputImageBGR[r][c][colorIndex] = OrgImageList[i + colorIndex][r  + sliceIndex * RowCutIndex][c]

                        # 只有中間那張的結果
                        if(colorIndex == 1):
                            if(CheckIfIsBorder(LabelImageList[i + colorIndex], r  + sliceIndex * RowCutIndex, c)):
                                ResultImageBGR[r][c] = 1
                            else:
                                ResultImageBGR[r][c] = 0
                    
                # 改一下結果
                # 邊界後面的結果為 2
                meet1 = False
                for c in range(min(col, CutIndex)):
                    if(meet1 == True and ResultImageBGR[r][c] == 1):
                        continue
                    elif(meet1 == True and ResultImageBGR[r][c] == 0):
                        ResultImageBGR[r][c] = 2
                    elif(meet1 == False and ResultImageBGR[r][c] == 1):
                        meet1 = True
                        
                    
                # 存出目錄
                SaveLocation = Input_FileNameList[i * rowOffset + sliceIndex]
                cv2.imwrite(SaveLocation, InputImageBGR)
                SaveLocation = Trainning_FileNameList[i * rowOffset + sliceIndex]
                cv2.imwrite(SaveLocation, ResultImageBGR)
        
    for i in reversed(deleteIndex):
        del Input_FileNameList[i]
        del Trainning_FileNameList[i]

In [12]:
BGRWithBorder()

100%|████████████████████████████████████████████████████████████████████████████████| 423/423 [25:33<00:00,  3.63s/it]


## 寫 Train.txt
---
InputImage LabelImage 

In [13]:
# 檔案名字從 Windows 轉道 Ubuntu
def FileNameWindowsToUbuntu(FileName):
    global WindowsDataLocation, UbuntuDataLocation
    return FileName.replace(WindowsDataLocation, UbuntuDataLocation)

In [14]:
TrainScript = ""
for i in range(len(Input_FileNameList)):
    TrainScript += FileNameWindowsToUbuntu(Input_FileNameList[i]) + " " + \
                FileNameWindowsToUbuntu(Trainning_FileNameList[i]) + "\n"

In [15]:
file = open(TrainTxtLocation, "w")
file.write(TrainScript)
file.close()